<a href="https://colab.research.google.com/github/londonsangongzi/colab_files/blob/main/test_finetuning_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wget

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Colab Codes/sgz_modules/'

sys.path.append(base_dir)
from sgzFinetuning import sgzFinetuning,sgzFinetuningConfig

Mounted at /content/drive


In [ ]:
import wget
import os
import pandas as pd

print('Downloading dataset...')
# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'
# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')
print('Done.')
# Unzip the dataset (if we haven't already)
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Get the lists of sentences and their labels.
#df = df.sample(256) #just for test
sentences = df.sentence.values
labels = df.label.values   

Done.
Number of training sentences: 8,551



In [ ]:
#sentences = sentences[:500]
#labels = labels[:500]

In [ ]:
print(type(sentences)) 

<class 'numpy.ndarray'>


In [ ]:
configs = []

#c = sgzFinetuningConfig(bOwnClassificationClass=False,learning_rate=1e-5,epochs=4)#原始基准,only cls+last embedding
#                                    0.49,0.79
#configs.append(c)
#c = sgzFinetuningConfig(learning_rate=1e-5,epochs=2)#自己模型基准   0.50,0.77/0.51,0.78
#configs.append(c)
c = sgzFinetuningConfig(train_ratio=0.1,learning_rate=1e-4,epochs=8,scheduler_type='ReduceLROnPlateau') 
configs.append(c)
"""
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,BN='before_activation',
                        pooling_way='concat_last4_hidden_states') 
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,BN='before_activation',pooling_token='words',
                        pooling_way='concat_last4_hidden_states',pooling_meanmax='mean') 
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,BN='before_activation',pooling_token='words',
                        pooling_way='concat_last4_hidden_states',pooling_meanmax='max') 
configs.append(c)
"""
#c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,BN='before_activation') 
#configs.append(c)  
"""       
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=8,BN='before_activation',pooling_token='words',
                        dim_of_hidden_layers=[64],
                        pooling_way='concat_last4_hidden_states',pooling_meanmax='concat_mean_max') 
configs.append(c)               #0.45,0.81
"""
#c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,DROPOUT=False)   #0.49,0.79
#configs.append(c)

"""
c = sgzFinetuningConfig(learning_rate=1e-5,
                        epochs=6,BN='before_activation',
                        pooling_token='words',
                        pooling_way='concat_last4_hidden_states')#0.48,0.8 
configs.append(c)                        
""" 
"""
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=10,BN='before_activation',
                        freeze_bert=('gradual_unfreezing',[[5],[4],[3],[2],[1],[0]]),
                        )
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=10,BN='before_activation',
                        freeze_bert=('gradual_unfreezing',[[5],[4],[3],[2],[1],[0]]),
                        pooling_token='words',pooling_way='concat_last4_hidden_states',
                        )
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=10,BN='before_activation',
                        freeze_bert=('gradual_unfreezing',[[5],[4],[3],[2],[1],[0]]),
                        pooling_token='words',pooling_way='concat_last4_hidden_states',
                        dim_of_hidden_layers=[64],
                        )
configs.append(c)
"""
"""
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=5,
                        freeze_bert=('gradual_unfreezing',[[5,4],[3,2],[1,0]]),
                        )
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,
                        freeze_bert=('gradual_unfreezing',[[5,4,3],[2,1,0]]),
                        )
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=4,
                        freeze_bert=('gradual_unfreezing',[[5,4,3,2,1,0]]),
                        )
configs.append(c)
"""
"""
c = sgzFinetuningConfig(learning_rate=5e-6,
                        epochs=16,BN='before_activation',
                        pooling_token='words',pooling_way='concat_last4_hidden_states',
                        dim_of_hidden_layers=[64],)#0.45,0.81
configs.append(c)
c = sgzFinetuningConfig(learning_rate=5e-6,
                        epochs=16,BN='before_activation',
                        pooling_token='words',pooling_way='concat_last4_hidden_states',
                        dim_of_hidden_layers=[256,64],)#0.45,0.80->0.44,0.80/0.45,0.81
configs.append(c)
"""
"""
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=6,BN='after_activation') #0.49,0.78(0.50,0.78->)0.51,0.79
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=6,BN='before_activation',DROPOUT=False) #0.46,0.79(0.48,0.79->)0.51,0.8
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,epochs=6,BN='after_activation',DROPOUT=False) #0.49,0.78/0.5,0.79
configs.append(c)
"""

#c = sgzFinetuningConfig(learning_rate=1e-5,pooling_way='concat_last4_hidden_states',epochs=6)
#configs.append(c)
#c = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[256],pooling_way='concat_last4_hidden_states',epochs=6)
#configs.append(c)
"""
c = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[256],epochs=6)#
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[64],epochs=6)#
configs.append(c)
c = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[256,64],epochs=6)#
configs.append(c)
"""


'\nc = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[256],epochs=6)#\nconfigs.append(c)\nc = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[64],epochs=6)#\nconfigs.append(c)\nc = sgzFinetuningConfig(learning_rate=1e-5,dim_of_hidden_layers=[256,64],epochs=6)#\nconfigs.append(c)\n'

In [ ]:
for config in configs:
  ft = sgzFinetuning(sentences,labels,config,random_seed=8)
  ft.train()

No GPU available, using the CPU instead.
  855 training samples
7,696 validation samples
 已耗时 0:08:29 ,预计 0:33:56

KeyboardInterrupt: ignored